In [1]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [2]:
import sys
import os

sys.path.insert(0, os.path.abspath(".."))
sys.path.append(os.path.abspath("../backend"))
sys.path.append(os.path.abspath(""))

In [3]:
from rdflib.plugins.stores.sparqlstore import SPARQLStore
from backend.ontology import Graph, OntologyManager, OntologyConfig
from backend.datasetmatcher import DatasetManager
from backend.explorative.explorative_support import GuidanceManager
from backend.explorative.llm_query import LLMQuery
from backend.explorative.topic_init import TopicInitator

from backend.eval_config import (
    DBPEDIA_CONFIGS,
    OMA_CONFIGS,
    UNIPROT_CONFIGS,
    BTO_CONFIGS,
    DNB_CONFIGS,
    EvalConfig,
)
base_path = "../data"
onto_path = f"{base_path}/hero-ontology/hereditary_clinical.ttl"


# graph = Graph().parse(onto_path, format="turtle")
# graph.bind("bto", "http://www.semanticweb.org/ontologies/2020/3/bto#")

# store = SPARQLStore(
#     "http://localhost:7200/repositories/dpedia",
#     method="POST_FORM",
#     params={"infer": False, "sameAs": False},
# )

# store = SPARQLStore(
#     "http://localhost:3030/dbpedia/query",
#     method="POST_FORM",
#     params={"infer": False, "sameAs": False},
# )
setup: EvalConfig = YAGO_CONFIGS[-1]
print("Setup for ", setup.name)
store = SPARQLStore(
    setup.sparql_endpoint,
    method="POST_FORM",
    params={"infer": False, "sameAs": False},
    retries=10,
)
graph = Graph(store=store)

config = OntologyConfig()

ontology_manager = OntologyManager(config, graph)
dataset_manager = DatasetManager(ontology_manager)
dataset_manager.initialise(glob_path="data/datasets/ALS/**/*.csv")

# ontology_manager.load_full_graph()


Setup for  BTO
-- Loading and merging datasets
0 datasets loaded


In [5]:

guidance_man = GuidanceManager(
    ontology_manager, conn_str=setup.conn_str, llm_model_id=setup.model_id
)
print(guidance_man.identifier)
llm_query = LLMQuery(guidance_man)
topic_init = TopicInitator(guidance_man)

53b0ad130b816dae48f9b2782cf10506b88a27a95fd38e0fdfafabe84dfb916a


In [6]:
topic_init.initate()

Initializing topics


Saving anonymous properties: 0it [00:00, ?it/s]
Enriching classes:  27%|██▋       | 140/511 [06:18<16:43,  2.70s/it]  


KeyboardInterrupt: 

In [ ]:
llm_query.initiate(n_queries=10, force=True)